Get history of data: https://docs.microsoft.com/en-us/azure/databricks/delta/delta-utility#python-1

Read older versions: https://docs.delta.io/latest/quick-start.html#read-older-versions-of-data-using-time-travel

In [0]:
from delta.tables import DeltaTable

In [0]:
# from pyspark.sql.types import StructType,StructField, StringType, IntegerType

# #Version 1
# data = [("James","","Smith","36636","M",3000),
#     ("Michael","Rose","","40288","M",4000),
#     ("Robert","","Williams","42114","M",4000),
#     ("Maria","Anne","Jones","39192","F",4000),
#     ("Jen","Mary","Brown","","F",-1)
#   ]

# schema = StructType([ \
#     StructField("firstname",StringType(),True), \
#     StructField("middlename",StringType(),True), \
#     StructField("lastname",StringType(),True), \
#     StructField("id", StringType(), True), \
#     StructField("gender", StringType(), True), \
#     StructField("salary", IntegerType(), True) \
#   ])
 
# df = spark.createDataFrame(data=data,schema=schema)
# df.write.mode("overwrite").save("/mnt/delta/employee")

# #Version 2
# data = [("Sacha","","Smith","36636","M",3000),
#     ("Dimitri","Rose","","40288","M",4000),
#     ("Raskolnikov","","Williams","42114","M",4000),
#     ("Macha","Anne","Jones","39192","F",4000),
#     ("Anton","Mary","Brown","","F",-1)
#   ]

# schema = StructType([ \
#     StructField("firstname",StringType(),True), \
#     StructField("middlename",StringType(),True), \
#     StructField("lastname",StringType(),True), \
#     StructField("id", StringType(), True), \
#     StructField("gender", StringType(), True), \
#     StructField("salary", IntegerType(), True) \
#   ])
 
# df = spark.createDataFrame(data=data,schema=schema)
# df.write.mode("overwrite").save("/mnt/delta/employee")

In [0]:
employees = spark.read.load("/mnt/delta/employee", format="delta")
employees.show()

+-----------+----------+--------+-----+------+------+
 firstname|middlename|lastname| id|gender|salary|
+-----------+----------+--------+-----+------+------+
Raskolnikov| |Williams|42114| M| 4000|
 Macha| Anne| Jones|39192| F| 4000|
 Dimitri| Rose| |40288| M| 4000|
 Sacha| | Smith|36636| M| 3000|
 Anton| Mary| Brown| | F| -1|
+-----------+----------+--------+-----+------+------+

In [0]:
employees_for_history = DeltaTable.forPath(spark, "/mnt/delta/employee")
display(employees_for_history.history())

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
1,2021-05-26T07:22:50.000+0000,808098371030088,nastasia.saby@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(142234619150852),0526-055223-sine958,0,WriteSerializable,false,"Map(numFiles -> 6, numOutputBytes -> 8552, numOutputRows -> 5)",null
0,2021-05-26T07:22:44.000+0000,808098371030088,nastasia.saby@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(142234619150852),0526-055223-sine958,null,WriteSerializable,false,"Map(numFiles -> 6, numOutputBytes -> 8487, numOutputRows -> 5)",null


In [0]:
first_version_of_employees = spark.read.option("versionAsOf", 0).format("delta").load("/mnt/delta/employee")
display(first_version_of_employees)

firstname,middlename,lastname,id,gender,salary
Maria,Anne,Jones,39192,F,4000
Robert,,Williams,42114,M,4000
Michael,Rose,,40288,M,4000
James,,Smith,36636,M,3000
Jen,Mary,Brown,,F,-1


In [0]:
second_version_of_employees = spark.read.format("delta").load("/mnt/delta/employee")
display(second_version_of_employees)

firstname,middlename,lastname,id,gender,salary
Raskolnikov,,Williams,42114,M,4000
Macha,Anne,Jones,39192,F,4000
Dimitri,Rose,,40288,M,4000
Sacha,,Smith,36636,M,3000
Anton,Mary,Brown,,F,-1
